# Очистка и подготовка данных

## (๑'ᵕ'๑)⸝* Упр. - Пример очистки данных

In [1]:
import pandas as pd

data = {'Клиент': ['A', 'B', 'C', 'D', 'E'],
        'MonthlyIncome': [5000, None, 7000, None, 9000]}
df = pd.DataFrame(data)
# Замена медианой
df_median = df.copy(deep=True)
df_median['MonthlyIncome'] = df_median['MonthlyIncome'].fillna(df_median['MonthlyIncome'].median())
print('Результат с заменой медианным значением:')
df_median

Результат с заменой медианным значением:


,Клиент,MonthlyIncome
0,A,5000.0
1,B,7000.0
2,C,7000.0
3,D,7000.0
4,E,9000.0


In [2]:
# Линейная интерполяция
df_lin = df.copy(deep=True)
df_lin['MonthlyIncome'] = df_lin['MonthlyIncome'].interpolate()
print('Результат с заменой при Линейной интерполяции:')
df_lin

Результат с заменой при Линейной интерполяции:


,Клиент,MonthlyIncome
0,A,5000.0
1,B,6000.0
2,C,7000.0
3,D,8000.0
4,E,9000.0


## (๑'ᵕ'๑)⸝*  Упр. - Очистка и подготовка данных
- Примените различные методы очистки данных к набору данных с проблемами качества.
- Сравните результаты и выберите наилучший подход.

In [3]:
import pandas as pd
import seaborn as sns

penguins = sns.load_dataset('penguins')
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [4]:
penguins.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


In [5]:
print('== Общая информация о датафрейме ==\n\
-------------------------------------')
penguins.info()

== Общая информация о датафрейме ==
-------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


Один из вариантов - просто удаление всех пропусков. С помощью  `dropna()`  удаляются все строки с пропусками.

In [6]:
penguins_dropna = penguins.copy(deep=True)
penguins_dropna = penguins_dropna.dropna()
print('== Простое удаление пропусков ==\n\
-------------------------------------')
penguins_dropna.info()


== Простое удаление пропусков ==
-------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 333 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            333 non-null    object 
 1   island             333 non-null    object 
 2   bill_length_mm     333 non-null    float64
 3   bill_depth_mm      333 non-null    float64
 4   flipper_length_mm  333 non-null    float64
 5   body_mass_g        333 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 20.8+ KB


In [7]:
df_clean = penguins.copy()
# Заполнение числовых столбцов медианой
for col in df_clean.select_dtypes(include='number'):
    df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# Заполнение категориальных столбцов модой
for col in df_clean.select_dtypes(include='object'):
    df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])

# Удаление дубликатов
df_clean = df_clean.drop_duplicates()

# Удаление выбросов по IQR только для числовых столбцов
numeric_cols = df_clean.select_dtypes(include='number').columns

Q1 = df_clean[numeric_cols].quantile(0.25)
Q3 = df_clean[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

# Маска строк без выбросов
mask = ~((df_clean[numeric_cols] < (Q1 - 1.5 * IQR)) | 
         (df_clean[numeric_cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# Финальный датафрейм без выбросов
df_clean_no_outliers = df_clean[mask]


In [8]:
df_clean_no_outliers.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.10,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.50,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.30,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,44.45,17.3,197.0,4050.0,Male
4,Adelie,Torgersen,36.70,19.3,193.0,3450.0,Female


Либо можем заполнить вычислямыми значениями. Например медианой или средним для числовых данных и модой для категориальных

In [9]:
df_clean = penguins.copy(deep=True)

# Заполняем числовые переменные медианой
for col in df_clean.select_dtypes(include='number'):
    df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# Заполняем категориальные переменные модой
for col in df_clean.select_dtypes(include='object'):
    df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])
df_clean.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.10,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.50,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.30,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,44.45,17.3,197.0,4050.0,Male
4,Adelie,Torgersen,36.70,19.3,193.0,3450.0,Female


In [10]:
print('== Заполнение пропусков медианой и модой ==\n')
df_clean.info()

== Заполнение пропусков медианой и модой ==

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     344 non-null    float64
 3   bill_depth_mm      344 non-null    float64
 4   flipper_length_mm  344 non-null    float64
 5   body_mass_g        344 non-null    float64
 6   sex                344 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


Дополнительно к заполнению можем добавить очистку данных от выбросов на основании расчета IQR для числовых столбцов.

Не всегда нужно просто удалять выбросы - так мы можем потерять ценные данные в других столбцах. Можно применять различные методы - например заменить выбросы на средним значением или применить метод clip, который заменить все что больше верхней расчетной границы на занчение этой границы. Аналогичным образом и с нижней.

Применение различных методов работы с выбросами зависит от каждой конкретной ситуации. Хотя заранее нам еж известно, что в данном датасете нет выбросов, тут ниже для примера я демонстрирую применения метода clip.

In [11]:
for col in df_clean.select_dtypes(include='number'):
    q1 = df_clean[col].quantile(0.25)
    q3 = df_clean[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    df_clean[col] = df_clean[col].clip(lower, upper)

print('== Удаление выбросов по IQR ==\n')
df_clean.describe()

== Удаление выбросов по IQR ==



,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,344.000000,344.000000,344.000000,344.000000
mean,43.925000,17.152035,200.892442,4200.872093
std,5.443792,1.969060,14.023826,799.696532
min,32.100000,13.100000,172.000000,2700.000000
25%,39.275000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


Для примера применения алгоритмов машинного обучения, возьмем KNNImputer

In [12]:
import numpy as np
from sklearn.impute import KNNImputer

df_knn = penguins.copy(deep=True)
num_cols = df_knn.select_dtypes(include=np.number).columns

imputer = KNNImputer(n_neighbors=3)
df_knn[num_cols] = imputer.fit_transform(df_knn[num_cols])

print('== Применение алгоритмов машинного обучения ==\n')
df_knn.info()

== Применение алгоритмов машинного обучения ==

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     344 non-null    float64
 3   bill_depth_mm      344 non-null    float64
 4   flipper_length_mm  344 non-null    float64
 5   body_mass_g        344 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [13]:
df_knn.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,344.000000,344.000000,344.000000,344.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.443643,1.969027,14.020657,799.613058
min,32.100000,13.100000,172.000000,2700.000000
25%,39.275000,15.600000,190.000000,3550.000000
50%,44.250000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


### Заключение

#### Сравнение каждого метода очистки данных:

1. **Удаление строк с пропущенными значениями:**
    - Плюсы: Простота, данные остаются чистыми.
    - Минусы: Потеря данных, особенно если пропущенных значений много.
2. **Заполнение средним/медианным значением:**
    - Плюсы: Сохранение всех строк, простота.
    - Минусы: Может исказить данные, особенно если пропущенные значения не случайны.
3. **Заполнение модой для категориальных столбцов:**
    - Плюсы: Сохранение всех строк, подходит для категориальных данных.
    - Минусы: Может исказить данные, если мода не репрезентативна.
4. **Удаление выбросов**
    - Плюсы: Полезно, когда есть выбросы, искажающие реальную картину распределения данных
    - Минусы: Не всегда понятно, что считать выбросами. Есть риск удалить значимую информацию и искадить картину
5. **Применение методов машинного обучения**
    - Плюсы: Часто могут дать результаты, лучще отрадающие реальную картину
    - Минусы: Применение далеко не всегда обосновано, замедляет обработку данных, требует обучение модели, при выбре неподходящего алгоритма может исказить результаты.
6. **Интерполяция:**
    - Плюсы: Сохранение всех строк, подходит для временных рядов или упорядоченных данных.
    - Минусы: Может быть неэффективным для данных без четкой структуры.

#### Выбор наилучшего подхода

- Если потеря данных не критична, и важно сохранить только полные данные, то удаление строк с пропущенными значениями может быть лучшим выбором. - но так редко когда происходит
- Если важно сохранить все строки и данные имеют числовую природу, то заполнение средним/медианным значением может быть подходящим.
- Для категориальных данных заполнение модой может быть более уместным.
Если данные имеют временную или упорядоченную структуру, то интерполяция может быть полезной.